In [35]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


load_dotenv()

llm = ChatOpenAI(
    model_name="gpt-4-turbo-preview",
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

In [36]:
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=3000,
    chunk_overlap=100,
)


loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)


embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)


chain.invoke(
    """
    3가지 질문에 대해 대답해줘
        1. Aaronson 은 유죄인가요? 의심가는 부분은 무엇인가요?
        2. 그가 테이블에 어떤 메시지를 썼나요?
        3. Julia 는 누구인가요?
    """
)

AIMessage(content='1. Aaronson이 유죄인지 여부에 대한 직접적인 언급은 없습니다. 그러나 문서에서는 "Oceania was at war with Eastasia. Oceania had always been at war with Eastasia. Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with."라고 언급되어 있습니다. 이는 파티의 공식 입장을 반영한 것일 수 있으나, 실제로 그들이 유죄인지, 아니면 파티가 조작한 것인지는 의심스러운 부분입니다. 특히, "He had never seen the photograph that disproved their guilt. It had never existed, he had invented it."라는 부분에서는 실제 증거의 존재 여부에 대한 의문이 제기됩니다.\n\n2. 테이블에 쓴 메시지는 "2+2=5"입니다. 이는 파티의 권력과 이데올로기를 수용하고, 객관적인 진실보다는 파티가 정의하는 \'진실\'을 받아들이는 것을 상징합니다.\n\n3. Julia는 문서에서 Winston과 관계를 맺는 여성입니다. 그녀는 Winston과 같이 반체제적인 생각을 가지고 있으며, 둘은 함께 파티의 규칙과 사회적 규범에 도전합니다. 그러나 나중에 둘 다 체포되어 고문을 받으며, 서로를 배신하게 됩니다. Julia에 대한 언급은 "He had seen her; he had even spoken to her. There was no danger in it. He knew as though instinctively that they now took almost no interest in his doings."와 같이 그들의 관계가 파티에 의해 파괴되었음을 보여줍니다.')